In [1]:
import random as rd

In [31]:
class Node():
    adjacent = []
    sample_q = []
    nearest_manhole = None
    timer = 999999999
    def __init__(self, name, status=0, samplable=False):
        self.name = name
        self.status = status
        self.samplable = samplable
    def __str__(self):
        temp = str(self.name) + " | STATUS: " + str(self.status) + " | ADJ: "
        temp2 = " | QUEUE: " + str(self.sample_q)
        return temp + str(self.adjacent) + " | MANHOLE: " + str(self.nearest_manhole) + temp2

In [32]:
def build_weighted_net(file_in):
    nodes = []
    counter = 0
    with open(file_in) as f:
        while(True):
            line = f.readline()
        
            # check for end of SIR layer
            if line == "\n":
                break
        
            new = Node(counter)
            split = line.split()
            adj = []
            assert len(split) % 2 == 0, "ERROR: each line in net cosntruction should have an even number of paramaters"
            for i in range(int(len(split)/2)):
                i = i * 2
                adj.append((int(split[i]), int(split[i + 1])))
            new.adjacent = adj
            nodes.append(new)
            counter += 1
            
        # sewer time
        sewer = {}
        while(True):
            line = f.readline()
            
            # check for end of manhole assignment
            if line == "\n":
                break
                
            split = line.split()
            name = ""
            adj = []
            assert len(split) % 2 == 0, "ERROR: each line in manhole construction should have an even number of paramaters"
            
            for i in range(int(len(split)/2)):
                i = i * 2
                name += split[i] + "-"
                adj.append((int(split[i]), int(split[i + 1])))
            name = name[:-1]
            new = Node(name)
            new.adjacent = adj
            new.samplable = True
            sewer[name] = new
            
            for i in adj:
                nodes[i[0]].nearest_manhole = (name, i[1])
                

        # creating sample queue
        for i in sewer.keys():
            for j in sewer[i].adjacent:
                queue = []
                for k in range(j[1]):
                    queue.append(0)
                nodes[j[0]].sample_q = queue
                
        # connecting manholes
        while(True):
            line = f.readline()
            
            # check for end of file
            if len(line) == 0:
                break
                
            split = line.split()
            assert len(split) == 3, "ERROR: each line connecting manholes should have exactly 3 paramaters"
            if split[1] != 'end':
                sewer[split[0]].nearest_manhole = (split[1], int(split[2]))
            else:
                end = Node('end')
                end.samplable = True
                sewer['end'] = end
                sewer[split[0]].nearest_manhole = ('end', int(split[2]))
                
        for i in sewer.keys():
            if i == 'end':
                break
            queue = []
            for j in range(sewer[i].nearest_manhole[1]):
                queue.append(0)
            sewer[i].sample_q = queue
            
    return nodes, sewer

In [33]:
net, sewer = build_weighted_net("sample_weights.txt")
for i in net:
    print(i)
    
for i in sewer.keys():
    print(sewer[i])

0 | STATUS: 0 | ADJ: [(1, 10), (2, 8)] | MANHOLE: ('0-1-2-3', 3) | QUEUE: [0, 0, 0]
1 | STATUS: 0 | ADJ: [(0, 10), (3, 4)] | MANHOLE: ('0-1-2-3', 1) | QUEUE: [0]
2 | STATUS: 0 | ADJ: [(0, 8), (3, 2)] | MANHOLE: ('0-1-2-3', 2) | QUEUE: [0, 0]
3 | STATUS: 0 | ADJ: [(1, 4), (2, 2)] | MANHOLE: ('0-1-2-3', 2) | QUEUE: [0, 0]
4 | STATUS: 0 | ADJ: [(5, 8)] | MANHOLE: ('4-5', 2) | QUEUE: [0, 0]
5 | STATUS: 0 | ADJ: [(4, 8), (6, 9)] | MANHOLE: ('4-5', 1) | QUEUE: [0]
6 | STATUS: 0 | ADJ: [(7, 2), (8, 5), (5, 9)] | MANHOLE: ('6-7-8-9-10', 1) | QUEUE: [0]
7 | STATUS: 0 | ADJ: [(6, 2), (10, 7)] | MANHOLE: ('6-7-8-9-10', 2) | QUEUE: [0, 0]
8 | STATUS: 0 | ADJ: [(6, 5), (9, 3)] | MANHOLE: ('6-7-8-9-10', 1) | QUEUE: [0]
9 | STATUS: 0 | ADJ: [(8, 3), (10, 2)] | MANHOLE: ('6-7-8-9-10', 2) | QUEUE: [0, 0]
10 | STATUS: 0 | ADJ: [(7, 7), (9, 2)] | MANHOLE: ('6-7-8-9-10', 1) | QUEUE: [0]
0-1-2-3 | STATUS: 0 | ADJ: [(0, 3), (1, 1), (2, 2), (3, 2)] | MANHOLE: ('4-5', 8) | QUEUE: [0, 0, 0, 0, 0, 0, 0, 0]
4-5 

In [58]:
def sample_simulate(net, sewer, rates, time):
    t = 0
    
    infected = []
    recovered = []
    samples = []
    sewer_nxt = []
    for i in net:
        if i.status == 1:
            infected.append(i)
        elif i.status == 2:
            recovered.append(i)
            
    while(t < time):
        t += 1
        
        new_infected = []
        for i in infected:
            
            # infecting others
            for j in i.adjacent:
                weight = j[1]
                j = net[j[0]]
                if j not in infected and j not in recovered and j not in new_infected:
                    # there is a chance to get infected
                    chance = rd.random() * weight
                    if chance > 1 - rates[0]:
                        j.status = 1
                        new_infected.append(j)
                        
            # recovering
            chance = rd.random()
            if chance < rates[1]:
                infected.remove(i)
                i.status = 2
                recovered.append(i)
                
        for i in new_infected:
            infected.append(i)
            
        # propagating sewer samples
        for i in net:
            i.sample_q.append(i.status % 2) # < --- change if you want more than 3 categories (i.e. SEIR instead of SIR)
        
        for i in sewer.keys():
            adj = sewer[i].adjacent
            marker = False
            for j in adj:
                popped = net[j[0]].sample_q.pop(0)
                if popped == 1:
                    marker = True
                    print("Sample recieved from node", j[0], "now node", sewer[i].name, "is samplable")
            sewer[i].status = int(marker)
            
        for i in sewer.keys():
            sewer[i].sample_q.append(sewer[i].status)
            
        for i in sewer.keys():
            if sewer[i].nearest_manhole != None:
                temp = sewer[i].nearest_manhole
                popped = sewer[i].sample_q.pop(0)
                sewer[temp[0]].status = max(sewer[temp[0]].status, popped)
                if popped == 1:
                    print("Sample recieved from node", i, "now node", sewer[temp[0]].name, "is samplable")
                
                
                
                
        print("TIME ---------------------------------", t)
        inf_string = ""
        for i in infected:
            inf_string += str(i.name) + " "
        rec_string = ""
        for i in recovered:
            rec_string += str(i.name) + " "
        print("INFECTED:", inf_string, "RECOVERED:", rec_string)
        sample_string = ""
        for i in sewer.keys():
            if sewer[i].status == 1:
                sample_string += i + " "
        print("VIABLE SAMPLE LOCATIONS:", sample_string, "\n")

In [59]:
net, sewer = build_weighted_net("sample_weights.txt")
net[3].status = 1
net[6].status = 1
for i in net:
    print(i)
for i in sewer.keys():
    print(sewer[i])
    
print("")
    
sample_simulate(net, sewer, [0.06, 0.3], 10)

0 | STATUS: 0 | ADJ: [(1, 10), (2, 8)] | MANHOLE: ('0-1-2-3', 3) | QUEUE: [0, 0, 0]
1 | STATUS: 0 | ADJ: [(0, 10), (3, 4)] | MANHOLE: ('0-1-2-3', 1) | QUEUE: [0]
2 | STATUS: 0 | ADJ: [(0, 8), (3, 2)] | MANHOLE: ('0-1-2-3', 2) | QUEUE: [0, 0]
3 | STATUS: 1 | ADJ: [(1, 4), (2, 2)] | MANHOLE: ('0-1-2-3', 2) | QUEUE: [0, 0]
4 | STATUS: 0 | ADJ: [(5, 8)] | MANHOLE: ('4-5', 2) | QUEUE: [0, 0]
5 | STATUS: 0 | ADJ: [(4, 8), (6, 9)] | MANHOLE: ('4-5', 1) | QUEUE: [0]
6 | STATUS: 1 | ADJ: [(7, 2), (8, 5), (5, 9)] | MANHOLE: ('6-7-8-9-10', 1) | QUEUE: [0]
7 | STATUS: 0 | ADJ: [(6, 2), (10, 7)] | MANHOLE: ('6-7-8-9-10', 2) | QUEUE: [0, 0]
8 | STATUS: 0 | ADJ: [(6, 5), (9, 3)] | MANHOLE: ('6-7-8-9-10', 1) | QUEUE: [0]
9 | STATUS: 0 | ADJ: [(8, 3), (10, 2)] | MANHOLE: ('6-7-8-9-10', 2) | QUEUE: [0, 0]
10 | STATUS: 0 | ADJ: [(7, 7), (9, 2)] | MANHOLE: ('6-7-8-9-10', 1) | QUEUE: [0]
0-1-2-3 | STATUS: 0 | ADJ: [(0, 3), (1, 1), (2, 2), (3, 2)] | MANHOLE: ('4-5', 8) | QUEUE: [0, 0, 0, 0, 0, 0, 0, 0]
4-5 